<a href="https://colab.research.google.com/github/norulwahab/nor_python_ml/blob/main/Malaria_model_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten, InputLayer,BatchNormalization, Input
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping, LearningRateScheduler, ModelCheckpoint,ReduceLROnPlateau
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
from tensorflow.keras.metrics import BinaryAccuracy, FalseNegatives,FalsePositives, TruePositives, TrueNegatives, Recall, Precision, AUC

In [2]:
dataset, dataset_info=tfds.load('malaria',with_info=True, as_supervised=True,shuffle_files=True, split=['train'])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incompleteMOJYOF/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
def splits(dataset, TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
  DATASET_SIZE = len(dataset)
  train_dataset = dataset.take(int(TRAIN_RATIO*DATASET_SIZE))

  val_test_dataset = dataset.skip(int(TRAIN_RATIO*DATASET_SIZE))
  val_dataset = val_test_dataset.take(int(VAL_RATIO*DATASET_SIZE))

  test_dataset = val_test_dataset.skip(int(VAL_RATIO*DATASET_SIZE))

  return train_dataset,val_dataset,test_dataset

In [4]:
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1

train_dataset,val_dataset,test_dataset = splits(dataset[0],TRAIN_RATIO,VAL_RATIO,TEST_RATIO)


##Data Visualization

In [5]:
#for i, (image, label) in enumerate(train_dataset.take(20)):
#  plt.subplot(4,5,i+1)
#  plt.imshow(image)
 # plt.title(dataset_info.features['label'].int2str(label))
  #plt.axis('off')

##Processing


In [6]:
IMAGE_SIZE=224
def resize_rescale(image,label):
  return tf.image.resize(image, (IMAGE_SIZE,IMAGE_SIZE))/255.0,label

In [7]:
train_dataset=train_dataset.map(resize_rescale)
val_dataset=val_dataset.map(resize_rescale)
test_dataset=test_dataset.map(resize_rescale)
# for image,label in train_dataset.take(1):
#   print(image,label)

In [8]:
train_dataset=train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [9]:
val_dataset=val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [10]:
test_dataset=test_dataset.batch(1)

##Model Creation and Training


In [11]:
class Custom_Dense(Layer):
  def __init__(self, output_units,activation):
    super(Custom_Dense, self).__init__()
    self.output_units = output_units
    self.activation = activation

  def build(self, input_features_shape):
    self.w = self.add_weight(shape = (input_features_shape[-1], self.output_units), initializer = 'random_normal', trainable =True)
    self.b = self.add_weight(shape = (self.output_units,), initializer = 'random_normal', trainable = True)

  def call(self,input_features):
    pre_output = tf.matmul(input_features,self.w) + self.b
    if (self.activation == 'relu'):
      return tf.nn.relu(pre_output)
    elif (self.activation == 'sigmoid'):
      return tf.math.sigmoid(pre_output)
    else:
      return pre_output

In [12]:
lenet_custom_model = tf.keras.models.Sequential([
                                      InputLayer(input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)),

                                      Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
                                      BatchNormalization(),
                                      MaxPooling2D(pool_size=2, strides=2),

                                      Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
                                      BatchNormalization(),
                                      MaxPooling2D(pool_size=2, strides=2),

                                      Flatten(),
                                      Custom_Dense(100, activation='relu'),
                                      BatchNormalization(),
                                      Custom_Dense(10, activation='relu'),
                                      BatchNormalization(),

                                      Custom_Dense(1, activation='sigmoid')
                                      ])
#lenet_custom_model.summary()

#Callbacks

In [13]:
class LossCallBack(Callback):
  def on_epoch_end(self, epoch, logs):
    print("\n For the Epoch number {}, the model has a Loss of {}".format(epoch+1,logs["loss"]))
  def on_batch_end(self, batch, logs):
    print("\n For the batch number {}, the model has a loss of {}".format(batch+1, logs))


##CSVLogger

In [14]:
callback_csv = CSVLogger(
    'csv.logger', separator=',', append=True
)

##EarlyStopping

In [15]:
es_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=7
)

##LearningRateSchedular

In [16]:
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr * tf.math.exp(-0.1)
lrshedular_callback = LearningRateScheduler(scheduler, verbose=1)

## ModelCheckpoint

In [58]:
model_check_points = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5',
    monitor = 'val_loss',
    verbose= 0,
    save_best_only = True,
    save_weights_only = True,
    mode='auto',
    save_freq='epoch',
)

## ReduceLROnPlateau

In [71]:
plateau_callback = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=2,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)

In [72]:
metrics =[BinaryAccuracy(name ='accuracy'), FalseNegatives(name ='fp'),FalsePositives(name ='fn'), TruePositives(name ='tp'),
          TrueNegatives(name ='tn'), Recall(name ='recall'), Precision(name ='precision'), AUC(name ='auc')]

In [73]:
lenet_custom_model.compile(optimizer=Adam(learning_rate=0.01), loss=BinaryCrossentropy(),metrics=metrics)

In [74]:
history= lenet_custom_model.fit(train_dataset,validation_data= val_dataset,epochs=13,verbose=1, callbacks=[plateau_callback])

Epoch 1/13
689/689 [==============================] - 19s 24ms/step - loss: 0.0243 - accuracy: 0.9916 - fp: 91.0000 - fn: 94.0000 - tp: 10931.0000 - tn: 10930.0000 - recall: 0.9917 - precision: 0.9915 - auc: 0.9994 - val_loss: 0.2679 - val_accuracy: 0.9445 - val_fp: 99.0000 - val_fn: 54.0000 - val_tp: 1297.0000 - val_tn: 1305.0000 - val_recall: 0.9291 - val_precision: 0.9600 - val_auc: 0.9753 - lr: 0.0100
Epoch 2/13
689/689 [==============================] - 16s 23ms/step - loss: 0.0261 - accuracy: 0.9912 - fp: 83.0000 - fn: 110.0000 - tp: 10940.0000 - tn: 10913.0000 - recall: 0.9925 - precision: 0.9900 - auc: 0.9991 - val_loss: 0.2292 - val_accuracy: 0.9387 - val_fp: 95.0000 - val_fn: 74.0000 - val_tp: 1293.0000 - val_tn: 1293.0000 - val_recall: 0.9316 - val_precision: 0.9459 - val_auc: 0.9776 - lr: 0.0100
Epoch 3/13
688/689 [============================>.] - ETA: 0s - loss: 0.0168 - accuracy: 0.9934 - fp: 68.0000 - fn: 77.0000 - tp: 10940.0000 - tn: 10931.0000 - recall: 0.9938 - prec

In [77]:
lenet_custom_model.evaluate(test_dataset)

2757/2757 [==============================] - 12s 3ms/step - loss: 0.4072 - accuracy: 0.9434 - fp: 65.0000 - fn: 91.0000 - tp: 1297.0000 - tn: 1304.0000 - recall: 0.9523 - precision: 0.9344 - auc: 0.9644


[0.4072275459766388,
 0.9434167742729187,
 65.0,
 91.0,
 1297.0,
 1304.0,
 0.9522760510444641,
 0.9344380497932434,
 0.9643754363059998]

In [19]:
lenet_custom_model.predict(test_dataset.take(1))[0][0]

1/1 [==============================] - 11s 11s/step


9.5001235e-07